# 라이브러리 로드

In [1]:
!pip install implicit
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit

# 데이터 로드

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')


In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,-251657396,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,-780140327,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,-1904213353,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,1518338663,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,-1769995873,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
data = df[['user_id','product_id','event_type']]

# 데이터 전처리

## view만 존재하는 유저 데이터 추출

1. cart, purchase 이력이 있는 유저들의 id값 받아오기
2. 위에서 얻은 cart, purchase 이력이 있는 유저들을 제외하기

In [6]:
drop_user_id = data.loc[data['event_type'] != 'view', 'user_id']

In [7]:
data = data.loc[~data['user_id'].isin(drop_user_id)].reset_index(drop=True)
data['event_type'].unique()

['view']
Categories (3, object): ['cart', 'purchase', 'view']

In [8]:
# event_type dtype 을 object로 변경

data['event_type'] = data['event_type'].astype('object')

## view 비율 구하기
- 해당 제품을 본 수 / 해당 유저가 전체 제품을 본 수

In [9]:
data.head()

,user_id,product_id,event_type
0,554748717,3900821,view
1,519107250,17200506,view
2,550050854,1307067,view
3,535871217,1004237,view
4,555447699,17300353,view


In [10]:
grouped = data.groupby(['user_id','product_id'])['event_type'].count()
grouped = grouped.reset_index()
grouped =grouped.rename(columns = {'event_type' : 'view_count'})

In [11]:
grouped

,user_id,product_id,view_count
0,33869381,7002639,1
1,64078358,10600284,1
2,183503497,22200103,1
3,184265397,6902133,2
4,184265397,6902303,2
...,...,...,...
15364922,566280663,1005127,2
15364923,566280676,13201002,1
15364924,566280697,2300307,1
15364925,566280780,15100003,1


In [12]:
total_event_type = data.groupby(['user_id'])['product_id'].count()
total_event_type = total_event_type.reset_index()
total_event_type = total_event_type.rename(columns={'product_id' : 'total_view'})

In [13]:
total_event_type

,user_id,total_view
0,33869381,1
1,64078358,1
2,183503497,1
3,184265397,6
4,195082191,1
...,...,...
2540827,566280663,2
2540828,566280676,1
2540829,566280697,1
2540830,566280780,1


In [14]:
data = grouped.merge(total_event_type, on='user_id')

In [15]:
data['view_ratio'] =  (data['view_count'] / data['total_view']) * 100
data.head()

,user_id,product_id,view_count,total_view,view_ratio
0,33869381,7002639,1,1,100.000000
1,64078358,10600284,1,1,100.000000
2,183503497,22200103,1,1,100.000000
3,184265397,6902133,2,6,33.333333
4,184265397,6902303,2,6,33.333333


# Item Table(product lookup) 테이블 만들기

In [16]:
product_lookup = df[['product_id','category_code','brand']].drop_duplicates('product_id').reset_index(drop=True).sort_values('product_id')
product_lookup.head()

,product_id,category_code,brand
151915,1000978,electronics.smartphone,NaN
8437,1001588,electronics.smartphone,meizu
85152,1001606,electronics.smartphone,apple
32556,1002042,electronics.smartphone,samsung
9400,1002062,electronics.smartphone,samsung


## Rating Matrix 만들기

In [17]:
num_user = data['user_id'].nunique()
num_item = data['product_id'].nunique()

num_user, num_item

(2540832, 159298)

In [18]:
users = list(np.sort(data['user_id'].unique()))
products = list(data['product_id'].unique())
ratio = list(data['view_ratio'])

rows = data['user_id'].astype('category').cat.codes
data['user_id_code'] = data['user_id'].astype('category').cat.codes

cols = data['product_id'].astype('category').cat.codes
data['product_id_code'] = data['product_id'].astype('category').cat.codes

len(users), len(products), len(ratio)

(2540832, 159298, 15364927)

In [19]:
user_item_matrix = sparse.csr_matrix((ratio, (rows, cols)), shape=(num_user, num_item))
user_item_matrix

<2540832x159298 sparse matrix of type '<class 'numpy.float64'>'
	with 15364927 stored elements in Compressed Sparse Row format>

# 추천 시스템 class 구현

In [20]:
from implicit.als import AlternatingLeastSquares
class MyALS():
  """
  implicit 라이브러리를 이용하여 필요한 기능을 담았습니다.

  Parameters
  ----------
  model : implicit 라이브러리의 AlternatingLeastSquares() 클래스로 생성된 인스턴스
  user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
  item_lookup : item(product)에 대한 정보를 담은 테이블
  data : User-Item Matrix를 만든 원본데이터

  """
  def __init__(self, model, user_item_matrix, item_lookup, data):

    self.model = model
    self.user_item_matrix = user_item_matrix
    self.item_lookup = item_lookup
    self.data = data
  

  def test_lookup(self):
    display(self.item_lookup.head())

  def fit(self, user_item_matrix):
    """
    행렬분해의 ALS를 이용하여 모델을 학습합니다.

    Parameters
    ----------
    user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
    """
    self.model.fit(user_item_matrix)
    
  def user_id_2_code(self, user_id):
    """
    입력받은 user_id를 User_Item_Matrix에 있는 user_id_code로 바꾸어주는 함수

    Parameters
    ----------
    user_id : 유저 ID

    """
    user_id_code = self.data.loc[self.data['user_id'] == user_id, 'user_id_code'].unique()[0]
    return user_id_code

  def product_id_2_code(self, product_id):
    """
    입력받은 product_id를 User-Item-Matrix에 있는 product_id_code로 바꾸어주는 함수

    Parameters
    ----------
    product_id : 상품 ID
    """
    product_id_code = self.data.loc[self.data['product_id'] == product_id, 'product_id_code'].unique()[0]
    return product_id_code

  def code_2_product_id(self, product_id_code):
    """
    입력받은 product_id_code를 User-Item-Matrix에 있는 product_id로 바꾸어주는 함수

    Parameters
    ----------
    product_id_code : 상품 ID code    
    """
    product_id = self.data.loc[self.data['product_id_code'] == product_id_code, 'product_id'].unique()[0]
    return product_id

  def get_recom_product(self, user_id, n = 10):
    """  
    user_id에 맞는 product를 n개 만큼 추천하여 데이터프레임 형태로 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    n : 추천 받게 될 item의 수
    """ 

    # user_id_2_code 함수를 이용하여 유저의 ID를 user_id_code로 변환합니다
    user_id_code = self.user_id_2_code(user_id)
  
    # model의 recommend를 이용하여 추천받는 제품의 id를 추출합니다.
    # 이때 추천 받는 제품의 id는 product_id가 아니라 product_id_code 입니다.
    recommended = self.model.recommend(user_id_code, self.user_item_matrix[user_id_code], N=n)[0]
    #결과를 담을 리스트를 초기화 합니다.
    results = []
    # 추천 받은 id를 돌면서 item_lookup 테이블에서 해당 product의 정보를 찾아 결과에 담습니다.
    for product_id_code in recommended:
      
      recommended_product_id = self.code_2_product_id(product_id_code)
      result = self.item_lookup.loc[self.item_lookup['product_id'] == recommended_product_id]
      results.append(result)
      
    return pd.concat(results)
  
  def get_user_topN_product(self,user_id,column, n = 20):
    """
    유저가 특정 기준값이 높은 제품 N개를 반환

    Parameters
    ----------
    user_id : 유저 ID
    column : 어떠한 값을 확인할 기준이 되는 컬럼
    n : 반환할 Item 수
    """

    product_ids = self.data.loc[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n]['product_id'].values
    product_values = self.data[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n][column].values
    results = []
    for i in product_ids:
      result = self.item_lookup.loc[self.item_lookup['product_id'] == i]
      results.append(result)
    frame = pd.concat(results)
    frame[column] = product_values

    return frame

  def get_explain(self, user_id, item_id, column):
    """
    사용자에게 제품이 추천된 이유를 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    item_id : Item(product) ID
    column : 확인할 컬럼
    """

    user_id_code = self.user_id_2_code(user_id)
    product_id_code = self.product_id_2_code(item_id)

    total_score, top_contributions, user_weights = self.model.to_cpu().explain(user_id_code, self.user_item_matrix, product_id_code)

    results = []
    categorys = []
    brands = []
    scores = []
    for id_, score_ in top_contributions:
      product_id = self.code_2_product_id(id_)
      result = self.data.loc[(self.data['product_id'] == product_id) & (self.data['user_id'] == user_id)][['user_id','product_id',column]]
      category = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'category_code'].unique()[0]
      brand = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'brand'].unique()[0]

      results.append(result)
      categorys.append(category)
      brands.append(brand)
      scores.append(score_)

    frame = pd.concat(results)
    frame['score'] = scores
    frame['category'] = categorys
    frame['brand'] = brands
    
    frame = frame[['user_id', 'product_id','category','brand',column, 'score']]
    return frame, total_score

# 추천하기

In [21]:
q1 = np.quantile(data['view_ratio'], 0.25)
q3 = np.quantile(data['view_ratio'], 0.75)
q1, q3

(2.666666666666667, 16.666666666666664)

In [22]:
data.loc[(data['view_ratio'] >= q1)&(data['view_ratio'] <= q3)].head()

,user_id,product_id,view_count,total_view,view_ratio,user_id_code,product_id_code
13,209714031,5500032,2,22,9.090909,8,19995
14,209714031,5500097,1,22,4.545455,8,20007
15,209714031,5500107,2,22,9.090909,8,20012
16,209714031,5500108,2,22,9.090909,8,20013
17,209714031,5500268,1,22,4.545455,8,20068


view_ratio가 2.6 ~ 16.6 사이에 있는 유저들 중에 한명을 골라 확인해보겠습니다.

In [23]:
model = MyALS(AlternatingLeastSquares(), user_item_matrix, product_lookup, data)

In [24]:
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

먼저 이 유저의 view_ratio가 높은 제품을 확인해보겠습니다.

In [25]:
user = 209714031
column = 'view_ratio'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_ratio
17706,5500007,NaN,panasonic,18.181818
13292,26900028,sport.trainer,housefit,13.636364
11638,5500032,NaN,panasonic,9.090909
34678,5500107,NaN,braun,9.090909
46741,5500108,NaN,panasonic,9.090909
5638,44500029,NaN,omabelle,9.090909
34615,5500097,NaN,philips,4.545455
117782,5500268,NaN,panasonic,4.545455
40097,8700249,appliances.personal.hair_cutter,galaxy,4.545455
14124,8700258,appliances.personal.hair_cutter,remington,4.545455


여러 카테고리가 비어있지만 카테고리 종류 중 hair_cutter나  
브랜드로는 panasonic, braun, philips 와 같은 브랜드를 본 비율이 높은것을 유추 해볼때 면도기 종류를 검색해본것으로 추측합니다.

이 유저에게 모델이 어떤 제품을 추천했는지 확인해보겠습니다.

In [26]:
model.get_recom_product(user)

,product_id,category_code,brand
15472,44500004,NaN,omabelle
6094,44500031,NaN,omabelle
62982,44500015,NaN,omabelle
5463,44500023,NaN,omabelle
143620,51800038,NaN,NaN
79630,44500016,NaN,omabelle
143626,49500007,NaN,NaN
54759,44500026,NaN,omabelle
6556,44500006,NaN,omabelle
143624,51800034,NaN,NaN


유저가 본 브랜드 중 omabelle 브랜드의 제품들을 다수 추천한 것으로 보입니다.

맨 위에 위치한 44500004 id를 가진 제품을 어떤 이유로 추천했는지 확인해보겠습니다.

In [27]:
explain_frame, total_score = model.get_explain(user, 44500004, column)
explain_frame

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


,user_id,product_id,category,brand,view_ratio,score
23,209714031,44500029,NaN,omabelle,9.090909,0.216218
24,209714031,52000065,NaN,NaN,4.545455,0.062076
12,209714031,5500007,NaN,panasonic,18.181818,0.000484
21,209714031,8700583,appliances.personal.hair_cutter,braun,4.545455,0.000366
13,209714031,5500032,NaN,panasonic,9.090909,0.000344
15,209714031,5500107,NaN,braun,9.090909,0.000085
14,209714031,5500097,NaN,philips,4.545455,0.000038
16,209714031,5500108,NaN,panasonic,9.090909,0.000027
17,209714031,5500268,NaN,panasonic,4.545455,0.000019
19,209714031,8700258,appliances.personal.hair_cutter,remington,4.545455,0.000015
